In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('NYC_Restaurants.csv', dtype=unicode)
df.head()

,Unnamed: 0,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,0,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,12/30/2014,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,12/30/2014,02/12/2015,Cycle Inspection / Re-inspection
1,1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,12/30/2014,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,12/30/2014,02/12/2015,Cycle Inspection / Re-inspection
2,2,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection
3,3,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection
4,4,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection


## **Q1. Create a unique name for each restaurant.**

_Make a copy of the dataframe. Create a new column called 'RESTAURANT' by concetenating DBA, BUILDING, STREET, ZIPCODE_

In [3]:
df_copy = df.copy()
df_copy['RESTAURANT'] = df['DBA'] + ' ' + df['BUILDING'] + ' ' + df['STREET'] + ' ' + df['ZIPCODE']

In [4]:
df_copy.head()

,Unnamed: 0,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,RESTAURANT
0,0,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,12/30/2014,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,12/30/2014,02/12/2015,Cycle Inspection / Re-inspection,WENDY'S 469 FLATBUSH AVENUE 11225
1,1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,12/30/2014,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,12/30/2014,02/12/2015,Cycle Inspection / Re-inspection,WENDY'S 469 FLATBUSH AVENUE 11225
2,2,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection,WENDY'S 469 FLATBUSH AVENUE 11225
3,3,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection,WENDY'S 469 FLATBUSH AVENUE 11225
4,4,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,Hamburgers,11/13/2014,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,16,NaN,NaN,02/12/2015,Cycle Inspection / Initial Inspection,WENDY'S 469 FLATBUSH AVENUE 11225


## **Q2. How many restaurants are included in the data?**

_DESCRIPTION DESCRIPTION BLAH_

## **Q3. How many chains are there?**

_DESCRIPTION DESCRIPTION BLAH_

## **Q4. Plot a bar graph of the top 20 most popular chains.**

_DESCRIPTION DESCRIPTION BLAH_

## **Q5. What fraction of all restaurants are chains?**

_DESCRIPTION DESCRIPTION BLAH_

## **Q6. Plot the number of non-chain restaurants in each boro.**

_DESCRIPTION DESCRIPTION BLAH_

## **Q7. Plot the _fraction_ of non-chain restaurants in each boro.**

_DESCRIPTION DESCRIPTION BLAH_

## **Q8. Plot the popularity of cuisines.**

_DESCRIPTION DESCRIPTION BLAH_

## **Q9. Plot the cuisines among restaurants which do not get cited for violations.**

_DESCRIPTION DESCRIPTION BLAH_

## **Q10. What cuisines tend to be the “cleanest”?**

_DESCRIPTION DESCRIPTION BLAH_

## **Q11. What are the most common violations in each borough?**

_Calculated the counts of each violation-boro combination by creating a dummy column with a value of 1 and collapsing it through a pivot table_

In [5]:
df_counts = DataFrame(copy(df), columns = df.columns.values)
df_counts['count'] = Series(repeat(1,len(df_counts)))

pivoted = pd.pivot_table(df_counts, index = 'BORO', columns = 'VIOLATION CODE', values='count', aggfunc=sum)
pivoted = pivoted.fillna(0)
pivoted = pivoted.drop('Missing')

pivoted.T.idxmax()

BORO
BRONX            10F
BROOKLYN         10F
MANHATTAN        10F
QUEENS           10F
STATEN ISLAND    10F
dtype: object

## **Q12. What are the most common violations per borough, after normalizing for the relative abundance of each violation?**

_Normalized the entire pivot table DataFrame by constructing a function that divides each violation count within a bourough by the total count for that violation. Returns the new pivot table._

In [6]:
def violationFrequency(df):
    pivot = pd.pivot_table(df_counts, index = "VIOLATION CODE", values = 'count', aggfunc=sum)
    pivot = pivot.T
    
    for i in df.columns.values:
        f = pivot[i]
        df[i] /= float(f)
    
    return df

In [7]:
violationFrequency(pivoted)
pivoted.T.idxmax()

BORO
BRONX            04I
BROOKLYN         02D
MANHATTAN        03E
QUEENS           03G
STATEN ISLAND    03C
dtype: object